In [1]:
import sys
import os
import transformers
import json
import torch
import numpy as np
from torch.utils.data import DataLoader , Dataset

/mnt/DGX01/Personal/slndir/evaluation/lib/python3.12/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
sys.path.append("../")

In [3]:
#loading in the JSON list of dictionaries
data = json.load(open("../data/geneset_dict.json"))

In [4]:
for item in data:
    print(item)
    break

{'Symbol': 'A1BG', 'Gene_ID': 'ENSG00000121410', 'Name': 'alpha-1-B glycoprotein', 'Aliases': ['A1B', 'ABG', 'GAB', 'HYST2477'], 'Other names': ['HEL-S-163pA', 'alpha-1B-glycoprotein', 'epididymis secretory sperm binding protein Li 163pA'], 'Description': 'The protein encoded by this gene is a plasma glycoprotein of unknown function. The protein shows sequence similarity to the variable regions of some immunoglobulin supergene family member proteins. [provided by RefSeq, Jul 2008].'}


In [5]:
sampler = DataLoader(data, batch_size=1, shuffle=True)
sample = next(iter(sampler))

In [6]:
print(sample)

{'Symbol': ['RNU7-155P'], 'Gene_ID': ['ENSG00000238759'], 'Name': ['RNA, U7 small nuclear 155 pseudogene'], 'Aliases': ['N/A'], 'Other names': ['N/A'], 'Description': ['N/A']}


In [7]:
query = """
Format this data into a question and answer pair. Use the information from some keys to make the question, and the information from remaining keys to make the answer. Format your question to resemble a user query. Ignore any objects with ['N/A'] as their value.
Provide the pair in this format: <question>question here</question><answer>answer here</answer>

"""

In [8]:
print(query + str(sample))


Format this data into a question and answer pair. Use the information from some keys to make the question, and the information from remaining keys to make the answer. Format your question to resemble a user query. Ignore any objects with ['N/A'] as their value.
Provide the pair in this format: <question>question here</question><answer>answer here</answer>

{'Symbol': ['RNU7-155P'], 'Gene_ID': ['ENSG00000238759'], 'Name': ['RNA, U7 small nuclear 155 pseudogene'], 'Aliases': ['N/A'], 'Other names': ['N/A'], 'Description': ['N/A']}


In [9]:
#load in llama model
from transformers import pipeline

model_id = "meta-llama/Llama-3.1-8B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": query},
    {"role": "user", "content": str(sample)},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': '<question>What is the name and description of the gene with the symbol RNU7-155P?</question>\n<answer>The name is RNA, U7 small nuclear 155 pseudogene. Unfortunately, no description is available.</answer>'}
